In [62]:
from re import L
import pandas as pd
import numpy as np
import datetime 
import os
import glob

path = 'C:/Users/nadia/OneDrive/Desktop/pirpython'
extension = 'csv'
os.chdir(path)
files_lst = glob.glob('*.{}'.format(extension))

for fl in files_lst:
    if fl[-9:] != 'utput.csv':
        namee = str(fl[:-4])
        print(fl, namee)
        lines_of_new_file = []
        data = pd.read_csv(fl)
        data['DATETIME']=pd.to_datetime(data['DATETIME'])

print("DATASET")
print(data, '\n')

PIRPYTHON.csv PIRPYTHON
DATASET
                DATETIME   C6H6   NO2  SO2   PM10  PM2.5    O3   CO
0    2020-01-01 00:00:00   9.98  57.0  8.0  133.0  112.0   8.0  NaN
1    2020-01-01 01:00:00  11.15  56.0  8.0  142.0  120.0  10.0  NaN
2    2020-01-01 02:00:00  11.71  53.0  9.0  197.0  175.0   6.0  NaN
3    2020-01-01 03:00:00   9.26  46.0  7.0  146.0  123.0   6.0  NaN
4    2020-01-01 04:00:00   9.68  45.0  8.0  133.0  117.0   5.0  NaN
...                  ...    ...   ...  ...    ...    ...   ...  ...
8779 2020-12-31 19:00:00   0.77  32.0  4.0   44.0   12.0  40.0  1.6
8780 2020-12-31 20:00:00   1.15  52.0  5.0   46.0   16.0  26.0  2.2
8781 2020-12-31 21:00:00   0.61  36.0  5.0   47.0   21.0  43.0  2.4
8782 2020-12-31 22:00:00   0.40  52.0  4.0   25.0    6.0  28.0  1.6
8783 2020-12-31 23:00:00   0.48  41.0  6.0   27.0    8.0  38.0  0.9

[8784 rows x 8 columns] 



In [63]:
# adding columns for later

data['date'] = data['DATETIME'].dt.date
data['O3rcount'] = data['O3'].rolling(8, min_periods=8).count()
data['COrcount'] = data['CO'].rolling(8, min_periods=8).count()

#for i in data['O3rcount'].dropna():
 #   if data['O3rcount'][i] < 8:
  #      data['O3'][i] = ''   

filter1 = data["O3rcount"] == 7
filter2 = data["O3rcount"] == 8
#data["O3"] = np.where(filter1 & filter2 , data['O3'], 0)

fil1 = data['O3rcount'] < 6
fil2 = data['O3rcount'].isnull()
data['O3'].mask(fil1 | fil2, 0)

data['O3roll'] = data['O3'].rolling(8).mean()
data['COroll'] = data['CO'].rolling(8).mean()

print("ADDED ROWS")
print(data.head(355),'\n')

ADDED ROWS
               DATETIME   C6H6   NO2   SO2   PM10  PM2.5    O3   CO  \
0   2020-01-01 00:00:00   9.98  57.0   8.0  133.0  112.0   8.0  NaN   
1   2020-01-01 01:00:00  11.15  56.0   8.0  142.0  120.0  10.0  NaN   
2   2020-01-01 02:00:00  11.71  53.0   9.0  197.0  175.0   6.0  NaN   
3   2020-01-01 03:00:00   9.26  46.0   7.0  146.0  123.0   6.0  NaN   
4   2020-01-01 04:00:00   9.68  45.0   8.0  133.0  117.0   5.0  NaN   
..                  ...    ...   ...   ...    ...    ...   ...  ...   
350 2020-01-15 14:00:00   2.03  37.0  16.0   30.0   16.0  20.0  0.1   
351 2020-01-15 15:00:00   2.09  42.0  17.0   35.0   18.0  21.0  0.2   
352 2020-01-15 16:00:00   1.97  44.0  15.0   34.0   23.0  20.0  0.3   
353 2020-01-15 17:00:00   1.76  53.0  13.0   31.0   20.0  15.0  0.6   
354 2020-01-15 18:00:00   1.73  56.0  13.0   32.0   21.0  14.0  0.6   

           date  O3rcount  COrcount  O3roll  COroll  
0    2020-01-01       NaN       NaN     NaN     NaN  
1    2020-01-01       NaN   

In [64]:
#second dataframe for calculations

grouped_single = data.groupby('date').agg({'SO2': ['count','mean'], 'PM10': ['count','mean'], 'PM2.5': ['count','mean'], 'O3roll': ['count','max'] , 'COroll': ['count','max']})
grouped_single.replace('', np.nan).count()
grouped_single.head(10)

SO2             PM10            PM2.5            O3roll          \
           count       mean count       mean count       mean  count     max   
date                                                                           
2020-01-01    24   7.750000    24  63.500000    24  49.375000     17  29.125   
2020-01-02    24  13.125000    24  32.916667    24  15.416667     24  73.625   
2020-01-03    24  13.708333    24  28.583333    11  23.545455     24  74.875   
2020-01-04    24   5.166667    24  43.333333    24  20.500000     24  84.500   
2020-01-05    24   6.708333    24  17.500000    24   8.250000     24  81.375   
2020-01-06    24  10.208333    10  20.900000    24   9.083333     24  50.125   
2020-01-07    24  18.000000    24  46.625000    24  25.333333     24  43.125   
2020-01-08    24  13.625000    24  48.458333    24  24.375000     24  84.750   
2020-01-09    24  17.041667     0        NaN    24  20.083333     24  94.000   
2020-01-10    24  13.333333    24  18.208333     0        NaN     24  66.250   

           COroll          
            count     max  
date                       
2020-01-01      0     NaN  
2020-01-02     24  2.6500  
2020-01-03     24  1.3500  
2020-01-04     24  0.4000  
2020-01-05     24  0.4625  
2020-01-06     24  1.3875  
2020-01-07     24  1.2375  
2020-01-08     24  1.5125  
2020-01-09     24  2.2500  
2020-01-10     24  1.8125

# C6H6

In [65]:
lines_of_new_file.append("ΒΕΝΖΟΛΙΟ")
lines_of_new_file.append(" ")

In [66]:
#Μέτρηση έγκυρων τιμών και έλεγχος πληρότητας

data.replace('', np.nan).count()
c = data['C6H6'].count()

lines_of_new_file.append(f"Οι συνολικές ωριαίες τιμές είναι: {data.shape[0]} ")
lines_of_new_file.append(f"Οι έγκυρες ωριαίες τιμές είναι: {c}")

if c > (0.9*data.shape[0]):
    lines_of_new_file.append("Ικανοποιείται ο έλεγχος πληρότητας (90% των ωριαίων τιμών)")
else:
    lines_of_new_file.append("Δεν ικανοποιείται ο έλεγχος πληρότητας(90% των ωριαίων τιμών)")

In [67]:
# Έλεγχος οριακής τιμής στο ημερολογιακό έτος (5 μg/m3)

rdch6 = round(data['C6H6'].mean(), 2)
lines_of_new_file.append(f"Μέση παρατηρούμενη ετήσια τιμή: {rdch6} μg/m3 ")

if data['C6H6'].mean() > 5:
    lines_of_new_file.append("Γίνεται υπέρβαση του ορίου (5 μg/m3)")
else:
    lines_of_new_file.append("Δεν παρατηρείται υπέρβαση του ορίου (5 μg/m3)")

# NO2

In [68]:
lines_of_new_file.append(" ")
lines_of_new_file.append("NO2")
lines_of_new_file.append(" ")

In [69]:
#Μέτρηση έγκυρων τιμών και έλεγχος πληρότητας (ωριαίο)

dno2 = data['NO2'].count()
lines_of_new_file.append(f"Οι συνολικές τιμές είναι {data.shape[0]}")
lines_of_new_file.append(f"Οι έγκυρες τιμές είναι {dno2}")

if data['NO2'].count() > (0.9*data.shape[0]):
    lines_of_new_file.append("Ικανοποιείται ο έλεγχος πληρότητας (90% των ωριαίων τιμών)")
else:
    lines_of_new_file.append("Δεν ικανοποιείται ο έλεγχος πληρότητας (90% των ωριαίων τιμών)")

In [70]:
# Ωριαία οριακή τιμή (200μg/m3 έως 18 φορές τον χρόνο)

if data.loc[data.NO2 > 200, 'NO2'].count() > 18 :
    lines_of_new_file.append("Γίνεται υπέρβαση του ορίου (200μg/m3 έως 18 φορές τον χρόνο)") 
else:
    dnoo22loc = data.loc[data.NO2 > 200, 'NO2'].count()
    lines_of_new_file.append(f"Δεν γίνεται υπέρβαση του ορίου στην ωριαία οριακή τιμή (200μg/m3) για πάνω από 18 φορές το έτος. Aριθμός υπερβάσεων: {dnoo22loc}")

In [71]:
# Οριακή τιμή στο ημερολογ. έτος = 40 μg/m3

if data['NO2'].mean() > 40:
    dno2mean = round(data['NO2'].mean(), 1)
    lines_of_new_file.append(f"Γίνεται υπέρβαση του ορίου (40 μg/m3): {dno2mean} μg/m3")
else:
    lines_of_new_file.append("Δεν παρατηρείται υπέρβαση του ορίου (40 μg/m3)")

# SO2

In [72]:
lines_of_new_file.append(" ")
lines_of_new_file.append("SO2")

In [73]:
#Μέτρηση έγκυρων τιμών και έλεγχος πληρότητας (ωριαίο)

so2c = data['SO2'].count()
lines_of_new_file.append(f"Οι συνολικές ωριαίες τιμές είναι {data.shape[0]}")
lines_of_new_file.append(f"Οι έγκυρες ωριαίες τιμές είναι {so2c}")

if data['SO2'].count() > (0.9*data.shape[0]):
    lines_of_new_file.append("Ικανοποιείται ο έλεγχος πληρότητας (90% των ωριαίων τιμών)")
else:
    lines_of_new_file.append("Δεν ικανοποιείται ο έλεγχος πληρότητας (90% των ωριαίων τιμών)")

In [74]:
# Ωριαία οριακή τιμή (350μg/m3 έως 24 φορές τον χρόνο)

if data.loc[data.SO2 > 350, 'SO2'].count() > 24 :
    lines_of_new_file.append("Γίνεται υπέρβαση του ορίου (350μg/m3 έως 24 φορές τον χρόνο)") 
else:
    lines_of_new_file.append("Δεν γίνεται υπέρβαση του ορίου στην ωριαία οριακή τιμή (350μg/m3) για πάνω από 24 φορές το έτος")

countso2 = data.loc[data.SO2 > 350, 'SO2'].count()
lines_of_new_file.append(f"Οι συνολικές υπερβάσεις ήταν: {countso2}")
lines_of_new_file.append(" ")

In [75]:
#Μέτρηση έγκυρων τιμών και έλεγχος πληρότητας για την ημερήσια οριακή τιμή (ημερήσιο)

groups_so2 = len(grouped_single['SO2']) - grouped_single['SO2']['count'].le(18).sum()
lines_of_new_file.append(f"Οι συνολικές ημερήσιες τιμές είναι {grouped_single.shape[0]}")
lines_of_new_file.append(f"Οι έγκυρες ημερήσιες τιμές είναι {groups_so2}")

if (len(grouped_single['SO2']) - grouped_single['SO2']['count'].le(18).sum()) > (0.9*data.shape[0]):
    lines_of_new_file.append("Ικανοποιείται ο έλεγχος πληρότητας")
else:
    lines_of_new_file.append("Δεν ικανοποιείται ο έλεγχος πληρότητας")

In [76]:
# Ημερήσια οριακή τιμή (125μg/m3 έως 3 φορές τον χρόνο)

if grouped_single['SO2']['mean'].gt(125).sum() > 3:
    lines_of_new_file.append("Παρατηρείται υπέρβαση του ορίου (125μg/m3 έως 3 φορές τον χρόνο)")
else:
    lines_of_new_file.append("Δεν παρατηρείται υπέρβαση του ορίου (125μg/m3 έως 3 φορές τον χρόνο)")

group_singl_mean = grouped_single['SO2']['mean'].gt(125).sum()
lines_of_new_file.append(f"Συνολικές υπερβάσεις που παρατηρήθηκαν: {group_singl_mean}")
lines_of_new_file.append(" ")

# PM10

In [77]:
lines_of_new_file.append("PM10")
lines_of_new_file.append(" ")

In [78]:
#Έλεγχος πληρότητας (και για τα 2)

lines_of_new_file.append(f"Συνολικές ημερήσιες τιμές: {grouped_single.shape[0]}")

checkpm10len =  len(grouped_single['PM10']) - grouped_single['PM10']['count'].le(18).sum()
lines_of_new_file.append(f"Οι έγκυρες ημερήσιες τιμές είναι {checkpm10len}")

if grouped_single['PM10']['mean'].count() > (0.9*grouped_single.shape[0]):
    lines_of_new_file.append("Ικανοποιείται ο έλεγχος πληρότητας (90% των ημερήσιων τιμών)")
else:
    lines_of_new_file.append("Δεν ικανοποιείται ο έλεγχος πληρότητας (90% των ημερήσιων τιμών)")

In [79]:
# Οριακή τιμή ημέρας = 50μg/m3 έως 35 φορές το έτος

if grouped_single['PM10']['mean'].gt(50).sum() > 35:
    lines_of_new_file.append("Παρατηρείται υπέρβαση του ορίου (50μg/m3 έως 35 φορές το έτος)")
else:
    lines_of_new_file.append("Δεν παρατηρείται υπέρβαση του ορίου (50μg/m3 έως 35 φορές το έτος)")
    
gr_sin_pm10 = grouped_single['PM10']['mean'].gt(50).sum()
lines_of_new_file.append(f"Συνολικές υπερβάσεις που παρατηρήθηκαν: {gr_sin_pm10}")  

In [80]:
# Οριακή τιμή έτους = 40 μg/m3 (με βάση τις ημερήσιες τιμές και όχι τις ωριαίες)

gr_sing_mean = round(grouped_single['PM10']['mean'].mean(),1)
lines_of_new_file.append(f"Η οριακή τιμή του έτους είναι {gr_sing_mean}")

if grouped_single['PM10']['mean'].mean() > 40:
    lines_of_new_file.append("Παρατηρείται παραβίαση του ορίου (40 μg/m3)")
else:
    lines_of_new_file.append("Δεν παρατηρείται υπέρβαση του ορίου (40 μg/m3)")

# PM2.5

In [81]:
lines_of_new_file.append(" ")
lines_of_new_file.append('PM2.5')
lines_of_new_file.append(" ")

In [82]:
# Έλεγχος πληρότητας για το έτος 

lines_of_new_file.append(f"Συνολικές τιμές: {grouped_single.shape[0]}")

gr_pm10_pm25 = len(grouped_single['PM10']) - grouped_single['PM2.5']['count'].le(18).sum()
lines_of_new_file.append(f"Οι έγκυρες τιμές είναι {gr_pm10_pm25}")

if len(grouped_single['PM10']) - grouped_single['PM2.5']['count'].le(18).sum() > (0.9*grouped_single.shape[0]):
    lines_of_new_file.append("Ικανοποιείται ο έλεγχος πληρότητας")
else:
    lines_of_new_file.append("Δεν ικανοποιείται ο έλεγχος πληρότητας")

In [83]:
# Οριακή τιμή έτους = 25 μg/m3 (με βάση τις ημερήσιες τιμές και όχι τις ωριαίες)

ch_pm25_mean = round(grouped_single['PM2.5']['mean'].mean(),1)
lines_of_new_file.append(f"Η οριακή τιμή του έτους είναι {ch_pm25_mean}")

if grouped_single['PM2.5']['mean'].mean() > 25:
    lines_of_new_file.append("Παρατηρείται παραβίαση του ορίου (25 μg/m3)")
else:
    lines_of_new_file.append("Δεν παρατηρείται υπέρβαση του ορίου 25 μg/m3")

# O3

In [84]:
lines_of_new_file.append(" ")
lines_of_new_file.append('O3')
lines_of_new_file.append(" ")

In [85]:
# Ημερήσιος μέσος όρος 8ώρου/ για να είναι τα δεδομένα πλήρη απαιτούνται 18 8ωροι μέσοι όροι για κάθε μέρα (75%)

lines_of_new_file.append(f"Οι συνολικές τιμές (ημερήσιος μέσος όρος 8ώρου) είναι {grouped_single.shape[0]}")
gr_le_o3roll = len(grouped_single['O3roll']) - grouped_single['O3roll']['count'].le(18).sum()
lines_of_new_file.append(f"Οι έγκυρες τιμές (18 8ωροι μέσοι όροι για κάθε μέρα (75%)) είναι {gr_le_o3roll}")

#print(grouped_single['O3roll']['count'].le(18).sum())
#pd.set_option('display.max_rows', 366)
#print(grouped_single['O3roll']['count'])

lines_of_new_file.append("different results from excel")

In [86]:
# Τιμή ορίου μέγιστου ημερήσιου 8ωρου: 120μg/m3 έως 25 φορές τον χρόνο σε διάστημα 3 χρόνων

gr_o3roll_max = round(grouped_single['O3roll']['max'].max(), 1)
lines_of_new_file.append(f"Η οριακή τιμή του έτους είναι {gr_o3roll_max} μg/m3")

if grouped_single['O3roll']['max'].max() > 120 and grouped_single['O3roll']['count'].gt(18): #25 
    lines_of_new_file.append("Παρατηρείται παραβίαση του ορίου (120μg/m3 έως 25 φορές τον χρόνο σε διάστημα 3 χρόνων)")
else:
    lines_of_new_file.append("Δεν παρατηρείται υπέρβαση του ορίου (120μg/m3 έως 25 φορές τον χρόνο σε διάστημα 3 χρόνων)")
    
grorollmaxsum = grouped_single['O3roll']['max'].gt(120).sum() 
lines_of_new_file.append(f"Συνολικές παραβιάσεις μέγιστου ημερήσιου 8ώρου: {grorollmaxsum}")

In [87]:
# Όριο συναγερμού: 240 μg/m3 για 3 συνεχόμενες ώρες

times = 0
size = len(data['O3'])
  
for i in range(size - 2):
  
    if data['O3'][i] > 240 and data['O3'][i + 1] > 240  and data['O3'][i + 2] > 240:
        times += 1
        
lines_of_new_file.append(f"Yπερβάσεις του ορίου συναργερμού (240 μg/m3 για 3 συνεχόμενες ώρες) που παρατηρήθηκαν: {times}")    

# CO

the data for this pollutant are not for the station of Pireus but for the station of Patision, Greece

In [88]:
lines_of_new_file.append(" ")
lines_of_new_file.append("CO")
lines_of_new_file.append(" ")

In [89]:
# Ημερήσιος μέσος όρος 8ώρου/ για να είναι τα δεδομένα πλήρη απαιτούνται 18 8ωροι μέσοι όροι για κάθε μέρα (75%)

lines_of_new_file.append(f"Οι συνολικές τιμές είναι {grouped_single.shape[0]}")
gr_coroll = len(grouped_single['COroll']) - grouped_single['COroll']['count'].le(18).sum()
lines_of_new_file.append(f"Οι έγκυρες τιμές είναι {gr_coroll}")

#print(grouped_single['COroll']['count'].le(18).sum())

pd.set_option('display.max_rows', 50)
#print(grouped_single['COroll']['count'])

lines_of_new_file.append("different results from excel")

In [90]:
# Τιμή ορίου μέγιστου ημερήσιου 8ωρου: 10mg/m3 

cccc = round(grouped_single['COroll']['max'].max(), 2)
lines_of_new_file.append(f"Η οριακή τιμή του έτους είναι {cccc} mg/m3")

if grouped_single['COroll']['max'].max() > 10 and grouped_single['COroll']['count'].gt(18):
    lines_of_new_file.append("Παρατηρείται παραβίαση του ορίου (10mg/m3)")
else:
    lines_of_new_file.append("Δεν παρατηρείται υπέρβαση του ορίου (10mg/m3)")
    
gr_croll_maxx = grouped_single['COroll']['max'].gt(10).sum()
lines_of_new_file.append(f"Συνολικές παραβιάσεις μέγιστου ημερήσιου 8ωρου: {gr_croll_maxx}")

In [91]:
nm = f'{namee}_Output.csv'
with open(f'{namee}_Output.txt', 'w',encoding="UTF-8") as f:
    for line in lines_of_new_file:
        #l = str(line.encode("utf-8"))
        f.write(line)
        f.write('\n')
#df = pd.read_csv(nm)
#df.to_excel(f'{namee}_Output.xlsx', 'Sheet1')
#os.remove(nm)